# Data cleaning

## Libraries and documents

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import pycountry #Library for country list
import us #Library for US states list

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("../reviews.csv", header=None)

## Duplicates and null values

The scraper driver created did not work properly and automatically scraped some duplicated reviews. 

In [4]:
df = df.drop_duplicates(subset=None, keep="first")
df = df.reset_index(drop=True)
df.head(5)

,0,1,2,3,4
0,"Date_of_experience: {""0"":""_355y0nZn"",""1"":""Dece...","Chicago, Illinois",50,Birthday week,An amazing once in a lifetime experience. You ...
1,Date of experience: June 2020,"Atlanta, Georgia",50,You must visit it!,"Whatever I say, it can't describe this archite..."
2,Date of experience: December 2019,"Ann Arbor, Michigan",50,Amazing Masterpiece,It is impossible to describe the greatness of ...
3,Date of experience: December 2019,"Limoux, France",50,Incredible building,"An absolutely stunning building, still in the ..."
4,Date of experience: November 2020,United Kingdom,50,Gaudi a true Genius!!,Gaudi was an Architectural Legend! Once this ...


In [5]:
df = df.rename(columns={0: "date", 1: "location", 2:"rating", 3:"title", 4:"review_body"})

## Cleaning the date column

In [6]:
# Deleting the prefix of the string
df["date"] = df["date"].str.lstrip("Date of experience: ").str.rstrip('"}')
df["date"] = df["date"].str.lstrip("_of_experience: {'0':'_355y0nZn','1':").str.lstrip('"0":"_355y0nZn","1":')

#Splitting the date column into month and yr and deleting old date column
df[["month", "year"]] = df["date"].str.split(pat=" ", expand=True)
df = df.drop("date", axis=1)

# In the december month, it only appears mber, so this needs to be fixed with the whole month name
df[df["month"]=="mber"] = df[df["month"]=="mber"].replace("mber", "December")

In [7]:
# Creating a datetime column for further analysis
df["date"] = df["year"].astype(str) + " " + df["month"]

# Deleting month and year columns
df = df.drop(columns=["month", "year"])

## Cleaning the location column

### Dealing with US states

In [8]:
#Creating a list with the US states (with us library)
states = [str(x).lstrip("<State:") for x in us.states.STATES]
# Creating an empty column with str "None"
df["country"] = "None"
# Replacing state name for EEUU in the country column
df["country"] = df['location'].apply(lambda x: "United States" if any(i in x for i in states) else None)

### Labeling the countries

In [9]:
# Spliting the str in location column into two columns
df["location1"] = df["location"].str.split(", ").str[-1]
df["location2"] = df["location"].str.split(", ").str[0]
# Creating list with countries
countries = [x.name for x in pycountry.countries]
countries.append("Vietnam")

In [10]:
# Looping through the column location1 and extracting country match from list
for country in df["location1"]:
    if country in countries:
        df["country"].loc[df["location1"]==country] = country

In [14]:
# Looping through the column location2 and extracting country match from list
for country in df["location2"]:
    if country in countries:
        df["country"].loc[df["location2"]==country] = country

In [15]:
# After cleaning the column there are arround 7000 places that were not labelled
# So they'll be labeled as "Others"
df["country"].isnull().sum()
# df["country"] = df.loc[df["country"].isnull()] = "Others"
df["country"] = df["country"].fillna("Others")

# Deleting useless columns (location, location1, location2)
df = df.drop(columns=["location", "location1", "location2"])

## Cleaning the rating column

In [16]:
# Removing zero at the end of the rating to be on scale 1-5
df["rating"] = [str(item).strip('0') for item in df["rating"]]

## Target labelling

When it comes to labelling the observations, two approaches are taken into consideration. Firstly, the human approach, which will consider the ratings of the review on TripAdvisor. Second, the machine approach, which will take into consideration the sentiment analyzed by the module `SentimentIntensityAnalyzer` from `nltk` library.

### The human approach: TripAdvisor rating 

In this first approach, the sentiment is based on feature `rating`: if `rating` is <= 2 the value is negative, if it is = 3 it is neutral and if the value is <=5 it is positive. 

In [17]:
# Changing the column name to int
df["rating"] = df["rating"].astype(int)

# Creating list of conditions
conditions = [(df['rating'] > 3), (df['rating'] == 3), (df['rating'] < 3)]

# create a list of the values we want to assign for each condition
values = ["positive", "neutral", "negative"]

# create a new column and use np.select to assign values to it using our lists as arguments
df['sentiment_value'] = np.select(conditions, values)

### The machine approach: `SentimentIntensityAnalyzer` from `nltk` library

In this second approach, the reviews are labelled by the compound score of the `SentimentIntensityAnalyzer`, which ranges from -1 being the most negative to 1 being the most positive. Therefore, a threshold of ±0,2 was set in order to create the neutral class label. 

In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [13]:
# Adding a polarity score column in the df
df['scores'] = df['review_body'].apply(lambda text: sid.polarity_scores(text))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['comp_score'] = df['compound'].['compound'].apply(lambda c: "positive" if c >= 0.2 else ("neutral" if c >= -0.2 and (c <= 0.2) else "negative"))

NameError: name 'sid' is not defined

In [34]:
#Saving file for further analysis
df.to_csv("../reviews_scores.csv")